# Acquire

In [1]:
import pandas as pd
import numpy as np
from env import get_db_url
import os

def get_log_data():

    '''
    This function acquires curriculum log data by accessing a SQL database and performing a SQL query to acquire
    selected curriculum log tables and columns and return it to a dataframe. Additionally, data is stored in a .csv 
    making it more efficient for future utilization of the same function.
    '''

    filename = 'curriculum_logs.csv'
    
    if os.path.isfile(filename):
        return pd.read_csv(filename)
    
    else:
        sql = """
        SELECT *
        FROM logs
        LEFT JOIN cohorts ON logs.cohort_id = cohorts.id
        """

        df = pd.read_sql(sql, get_db_url('curriculum_logs'))

        df.to_csv(filename)


# Prepare

In [2]:
import acquire

def prepare_logs(use_cache=True):
    """This function takes in the DataFrame from the get_log_data function located in the acquire file.
    Args: none. 
    Columns dropped: 'slack', 'id', and 'deleted_at'.
    Columns renamed: mapped values for program type to integers in 'program_id' column.
    Columns converted: 'start_date', 'end_date', 'created_at', 'updated_at' converted to DTG format.
    Columns concat: 'date' and 'time', converted to DTG
    Returns: prepared DF, and CSV named 'codeup_logs.csv'
      """
      #use local cache from CSV if available
    filename = "codeup_logs.csv"
    if os.path.isfile(filename) and use_cache:
        return pd.read_csv(filename)
    # acquire the data
    df = df = acquire.get_log_data()
    # drop unnecessary columns
    df = df.drop(columns=(['slack', 'id', 'deleted_at', 'Unnamed: 0']))
    # map programs to program ids
    df.program_id = df.program_id.map({1.0:'full_stack_php', 
    2.0:'full_stack_java', 3.0:'data_science', 4.0:'front_end_programming'})
    # convert dates to DTG
    dates = ['start_date', 'end_date', 'created_at', 'updated_at']
    for col in dates:
        df[col] = pd.to_datetime(df[col])
    # change cohort names to lower case
    df.name = df.name.str.lower()
    # convert date-time to DTG
    df['date_time'] = df.date + " " + df.time
    df.date_time = pd.to_datetime(df.date_time)
    # drop unnecessary columns
    df = df.drop(columns=(['date', 'time']))
    # add 'to_csv'
    df.to_csv(filename, index=False)
    return df


def df_q2_eda(df):
    '''
    Creates a new dataframe for question 2,
    "Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?"
    by dropping rows that aren't cohorts and renaming columns.
    '''
        
    # Removes all rows with null values
    df = df.dropna()
    
    # Drops rows that are staff under the 'name' column 
    df = df[(df['name'] != 'staff')] 

    # Renames specified columns
    df2 = df.copy()
    df2 = df2.rename(columns={'name': 'cohorts', 'program_id': 'programs'})
    return df2


def get_q6_eda_df():
    '''This function converts the column types in the CSV from object to the correct type. Using the CSV cached locally results in dates saved as 'object; instead of datetimes'''
    
    df = prepare_logs()
    df.date_time = pd.to_datetime(df.date_time)
      # convert dates to DTG
    dates = ['start_date', 'end_date', 'created_at', 'updated_at']
    for col in dates:
        df[col] = pd.to_datetime(df[col])
        # drop unnecessary columns
    return df